In [ ]:
import json
import urllib.request
import time

reps = ['arogozhnikov', 'maryszmary', 'lizaku', 'nevmenandr', 'ancatmara', 'JelteF']
# Если захочется попробовтаь на других пользователях:
    #, 'elmiram', 'maryszmary', 'lizaku', 'nevmenandr', 'ancatmara', 'arogozhnikov', 'gvanrossum']

# Функция, которая помогает при работе с репозиториями
def forall(rep, page):
    # Простите, мне показалось, что ввести свой токен в двух местах (ещё в fifth) будет удобнее и быстрее.
    token = ''
    url = 'https://api.github.com/users/%s/repos?page=%s&per_page=100' % (rep, page)
    headers = {'apikey': 'token %s' % token}
    request = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(request)
    text = response.read().decode('utf-8')
    data = json.loads(text)
    # возвращает переменную с json-объектом
    return data

In [ ]:
# Список репозиториев и их описаний
def first():
    rep = input('Выберите один из предложенных репозиториев:\n' + '\n'.join(reps) + '\n\nВведите его название: ')
    if rep not in reps:
        print('Такого репозитория нет в списке.')
        first()
    else:
        print('\nВы выбрали репозиторий:', rep, '\n')
        # Перелистывание страниц
        page = 1
        data = forall(rep, page)
        while len(data) == 100:
            page += 1
            data += forall(rep, page)
        print('\nВот список его(ее) репозиториев (%s):\n' % len(data))
        for i in data:
            print(i["name"],i["description"],sep=': ')
    return data

In [1]:
# Языки в репозитории
def second(data):
    res = {}
    lang = {}
    for i in data:
        res[i['name']] = i['language']
    for k in res.values():
        if k is not None:
            if k in lang:
                lang[k] += 1
            else:
                lang[k] = 1
    ks = []
    for key in lang.keys():
        ks.append(key)
    print('\nВ заданном репозитории использованы языки:', ', '.join(ks), '\n')
    for key, value in lang.items():
        if value % 10 == 1 and value != 11:
            print('Язык {} используется в {} репозитории.'.format(key, value))
        else:
            print('Язык {} используется в {} репозиториях.'.format(key, value))

In [ ]:
# Подсчет репозиториев у каждого пользователя
def third():
    long = {}
    data = list()
    for rep in reps:
        page = 1
        long[rep] = 0
        data = forall(rep, page)
        while len(data) == 100:
            long[rep] += len(data)
            page += 1
            data = forall(rep, page)
            time.sleep(2)
        else:
            long[rep] += len(data)
    print('\nИз списка больше всего репозиториев у:')
    for key, value in long.items():
        if value == max(long.values()):
            print(key, '(%s)' % value)

In [ ]:
# Просмотр языков каждого пользователя и добавление их в словарь
def forth():
    lnggs = {}
    for rep in reps:
        page = 1
        data = forall(rep, page)
        while len(data) == 100:
            page += 1
            for i in data:
                if i['language'] is not None:
                    if i['language'] in lnggs:
                        lnggs[i['language']] += 1
                    else:
                        lnggs[i['language']] = 1
            data = forall(rep, page)
        else:
            for i in data:
                if i['language'] is not None:
                    if i['language'] in lnggs:
                        lnggs[i['language']] += 1
                    else:
                        lnggs[i['language']] = 1
    for key, value in lnggs.items():
        if value == max(lnggs.values()):
            print('\nСамый популярный язык среди пользователей из списка:', key, '(%s).' % value)

In [2]:
# Подсчет подписчиков каждого пользователя
def fifth():
    fllw = {}
    # Пожалуйста, введите свой токен еще раз
    token = ''
    for rep in reps:
        fllw[rep] = 0
        page = 1
        url = 'https://api.github.com/users/%s/followers?page=%s&per_page=100&access_token=%s' \
              % (rep, page, token)
        response = urllib.request.urlopen(url)
        text = response.read().decode('utf-8')
        data = json.loads(text)
        while len(data) == 100:
            fllw[rep] += len(data)
            time.sleep(1)
            page += 1
            # Ссылку приходится обновлять в цикле, т.к. значение page неизменно
            url = 'https://api.github.com/users/%s/followers?page=%s&per_page=100&access_token=%s'\
              % (rep, page, token)
            response = urllib.request.urlopen(url)
            text = response.read().decode('utf-8')
            data = json.loads(text)
        else:
            fllw[rep] += len(data)
    print('\nБольше всего подписчиков у:')
    for key, value in fllw.items():
        if value == max(fllw.values()):
            print(key, '(%s)' % value)

In [ ]:
def main():
    second(first())
    third()
    forth()
    fifth()

if __name__=="__main__":
    main()